In [1]:
import os
import xlrd
import pandas as pd
import numpy as np
import sys
#import gensim
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer, WordNetLemmatizer
import matplotlib.pyplot as plt
import seaborn as sns
import time
from scipy.stats import randint as sp_randint
# machine learning
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
import scipy.stats as st
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition, pipeline, metrics, grid_search
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
import random

random.seed(9001)

porter_stemmer = PorterStemmer()
stemmer = SnowballStemmer("english")
stop_words = stopwords.words('english')

t0 = time.time()


os.chdir('C:\\Users\\swaroop\\Desktop\\interviews\\quantiphi')

C:\Users\swaroop\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\swaroop\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# Read the data from excel to a pandas dataframe 
xl = pd.ExcelFile("Training Sheet.xlsx")

train = xl.parse(xl.sheet_names[0])
#print train.head()

xl = pd.ExcelFile("Scoring Sheet.xlsx")


In [3]:
#Create features for the dataset
train['same_name_flag'] = train['name'] == train['display_name']

train['contains_year2'] = train["name"].apply(lambda x: 1 if '(' in str(x) else 0)


train['same_name_flag']  = train['same_name_flag'].astype(int)

train['name_length'] = train["name"].apply(lambda x: len(str(x)))

train['name_words_len'] = train["name"].apply(lambda x: len(str(x).strip().split()))
    

In [4]:
#Create more features for the dataset

train['board_rating_reason'] = train['board_rating_reason'].str.replace(',','')
train['board_rating_reason'] = train['board_rating_reason'].str.replace('.','')
train['board_rating_reason'] = train['board_rating_reason'].str.replace('/',' ')
train['board_rating_reason'] = train['board_rating_reason'].str.replace(';','')
train['board_rating_reason'] = train['board_rating_reason'].str.replace('(','')
train['board_rating_reason'] = train['board_rating_reason'].str.replace(')','')
#train['board_rating_reason'] = train['board_rating_reason'].str.replace('-',' ')
train['board_rating_reason'] = train['board_rating_reason'].str.lower()
stop = stopwords.words('english')

train['board_rating_reason'] = train['board_rating_reason'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))



train['name'] = train['name'].str.replace(',','')
train['name'] = train['name'].str.replace('.','')
train['name'] = train['name'].str.replace('/',' ')
train['name'] = train['name'].str.replace(';','')
train['name'] = train['name'].str.replace('(','')
train['name'] = train['name'].str.replace(')','')
#train['board_rating_reason'] = train['board_rating_reason'].str.replace('-',' ')
train['name'] = train['name'].str.lower()

train['name_wno_sw'] = train['name'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop)]))

train['name_wno_sw_length'] = train["name_wno_sw"].apply(lambda x: len(str(x)))
train['name_words_wno_sw_len'] = train["name_wno_sw"].apply(lambda x: len(str(x).strip().split()))


train['brr_TOKENIZED']=train['board_rating_reason'].apply(lambda x : filter(None,x.split(" ")))
train['brr_stemmed']=train['brr_TOKENIZED'].apply(lambda x : [porter_stemmer.stem(y) for y in x])
train['brr_stemmed_sentence']=train['brr_stemmed'].apply(lambda x : " ".join(x))



print train['brr_stemmed_sentence'][:5]


train = train.drop(['name_wno_sw', 'brr_TOKENIZED', 'brr_stemmed', 'board_rating_reason',], axis=1)




0    intens epic battl sequenc warfar sensual langu...
1          sequenc intens action violenc frighten imag
2    intens prolong sequenc sci-fi action violenc m...
3                                                gener
4    intens sequenc action adventur violenc frighte...
Name: brr_stemmed_sentence, dtype: object


In [5]:
train['first_char'] = train['name'].str.lower().str[0].astype(str)
train['last_char'] = train['name'].str.lower().str[-1].astype(str)


In [6]:
#convert categorical to numerical and tf-idf for the text columns

first_char_dummies_train = pd.get_dummies(train['first_char'], prefix='f_char_')

last_char_dummies_train = pd.get_dummies(train['last_char'], prefix='l_char_')

genre_dummies_train  = pd.get_dummies(train['genre'])

source_dummies_train  = pd.get_dummies(train['source'])

mbrdn_dummies_train  = pd.get_dummies(train['movie_board_rating_display_name'])

language_dummies_train  = pd.get_dummies(train['language'])

pmd_train  = pd.get_dummies(train['production_method'])

creative_type_train  = pd.get_dummies(train['creative_type'])

mrpdn_train  = pd.get_dummies(train['movie_release_pattern_display_name'])

#tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english') # , ngram_range=(1,2)
tf_vectorizer = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 5), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

train_b_rating_reason = tf_vectorizer.fit_transform(train.brr_stemmed_sentence)

train_vocab = pd.DataFrame(train_b_rating_reason.todense(), columns=tf_vectorizer.get_feature_names())


name_tf_vectorizer = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 5), use_idf=1,smooth_idf=1,sublinear_tf=1, stop_words = 'english')

train_name_data = name_tf_vectorizer.fit_transform(train.name.values.astype('U'))


train_name_vocab = pd.DataFrame(train_name_data.todense(), columns=name_tf_vectorizer.get_feature_names())

train_name_vocab.columns = ['name_col_f_' + str(col)  for col in train_name_vocab.columns]


In [7]:
# join the dummies created above

train_df = pd.concat([train, train_vocab, train_name_vocab, first_char_dummies_train, last_char_dummies_train, genre_dummies_train, source_dummies_train, mbrdn_dummies_train, language_dummies_train, pmd_train, creative_type_train, mrpdn_train], axis=1)


In [8]:
#drop unnecessary columns, these columns won't be useful in analysis and prediction

train_df = train_df.drop(['id', 'name', 'first_char', 'last_char', 'total', 'display_name', 'genre','source','movie_board_rating_display_name', 'language', 'production_method', 'creative_type', 'movie_release_pattern_display_name', 'brr_stemmed_sentence'], axis=1)


time.time()-t0

0.6590001583099365

In [10]:
#Split the data to train and test 

train_df["Category"] = train['Category'].apply(lambda x : x+1 if x%2==1 else x)
train_df.loc[train_df['Category'] == 10, 'Category'] = 8
print train_df["Category"].value_counts()

X_train = train_df.loc[train_df.production_year != 2011]
X_test = train_df.loc[train_df.production_year == 2011]
Y_train = X_train["Category"]
Y_test = X_test["Category"]

X_train  = X_train.drop(["Category", "production_year"],axis=1)
X_test  = X_test.drop(["Category", "production_year"],axis=1)

X_train.shape


4    451
2    415
6    212
8    118
Name: Category, dtype: int64


(986, 1256)

In [11]:
#Logistic Regression

logistic = LogisticRegression(random_state=1)

pca = decomposition.PCA()
pipe = Pipeline(steps=[('pca', pca), ('logistic', logistic)])

# Plot the PCA spectrum
pca.fit(X_train)

n_components = [20, 40, 64, 100]
Cs = np.logspace(-4, 4, 3)
a = 'ovr'
# Parameters of pipelines can be set using ‘__’ separated parameter names:
estimator = GridSearchCV(pipe,
                         dict(pca__n_components=n_components,
                              logistic__C=Cs, 
                              logistic__random_state=[None],
                              logistic__intercept_scaling=[1.0],
                              logistic__tol=[0.00001],
                              logistic__dual=[True],
                              logistic__multi_class=['ovr'],
                            logistic__class_weight =['balanced']))
estimator.fit(X_train, Y_train)

best_model = estimator.best_estimator_
print best_model    
    
print(" Test Accuracy: %.2f%%" % (accuracy_score(Y_test, best_model.predict(X_test)) * 100.0))
print(" Train Accuracy: %.2f%%" % (accuracy_score(Y_train, best_model.predict(X_train)) * 100.0))    


Pipeline(steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=100, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('logistic', LogisticRegression(C=1.0, class_weight='balanced', dual=True,
          fit_intercept=True, intercept_scaling=1.0, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=1e-05, verbose=0, warm_start=False))])
 Test Accuracy: 59.52%
 Train Accuracy: 63.49%


In [12]:
#One Vs Rest Classifier

OVsR = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, Y_train)

Y_pred = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, Y_train).predict(X_train)

print OVsR.score(X_test, Y_test)

#confusion_matrix(Y_train, Y_pred)

0.471428571429


In [12]:
# SVC

svm = SVC(decision_function_shape='ovr', random_state=1)
#svm.fit(X_train, Y_train)

#print svm.score(X_test, Y_test)

#print svm.predict(X_test)

param_dist = {"C":  [1, 10, 100, 200, 1000, 10000],
              #"gamma": st.expon(scale=.1),
              "kernel": ['rbf', 'linear'],
              'class_weight':['balanced'],
              'tol':[0.01]}

random_search = RandomizedSearchCV(svm, param_distributions=param_dist, cv=4, scoring="accuracy", n_jobs=-1)
                                   #n_iter=n_iter_search)

random_search.fit(X_train, Y_train)

best_est = random_search.best_estimator_

print best_est

print(" Test Accuracy: %.2f%%" % (accuracy_score(Y_test, best_est.predict(X_test)) * 100.0))
print(" Train Accuracy: %.2f%%" % (accuracy_score(Y_train, best_est.predict(X_train)) * 100.0))

SVC(C=100, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=1, shrinking=True, tol=0.01,
  verbose=False)
 Test Accuracy: 31.90%
 Train Accuracy: 55.88%


In [13]:
#Random Forest
random.seed(9001)

clf = RandomForestClassifier()


param_dist = {"max_depth": [3, 5, 6, 7, 8, 9],
              "max_features": [6, 10, 15, 20, 30, None],
              "min_samples_split": [2, 5, 10, 15, 100,120],
              "min_samples_leaf": [1, 2, 4, 5, 6, 7],
              "class_weight": ['balanced'],
              "n_estimators": [100, 120, 150,250,300],
               "oob_score": [True],
                "random_state": [9001]}

# run randomized search
#n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, cv=8, scoring='accuracy',  n_jobs=-1)
                                   #n_iter=n_iter_search)


random_search.fit(X_train, Y_train)


best_est = random_search.best_estimator_
print best_est

print(" Test Accuracy: %.2f%%" % (accuracy_score(Y_test, best_est.predict(X_test)) * 100.0))
print(" Train Accuracy: %.2f%%" % (accuracy_score(Y_train, best_est.predict(X_train)) * 100.0))

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=5, max_features=15,
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=120,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=True, random_state=9001, verbose=0, warm_start=False)
 Test Accuracy: 34.76%
 Train Accuracy: 42.60%


In [14]:
importances = best_est.feature_importances_

feat_labels = list(X_train.columns.values)

# Print the name and gini importance of each feature
#for feature in zip(feat_labels, clf.feature_importances_):
#    print(feature)
    
sfm = SelectFromModel(best_est, threshold=0.0005)

# Train the selector
sfm.fit(X_train, Y_train)

cols = []

for feature_list_index in sfm.get_support(indices=True):
    print(feat_labels[feature_list_index])


movie_sequel
name_length
name_words_len
name_wno_sw_length
name_words_wno_sw_len
action
action brief languag
action languag
action rude
action rude humor
action scari imag
action sexual
action sexual content
action violenc
action violenc brief
action violenc frighten
action violenc frighten imag
action violenc languag
action violenc mild
adventur
adventur violenc
art action
battl
battl sequenc
bloodi
bloodi violenc
brief
brief drug materi
brief mild languag
brief sensual
brief strong
brief strong languag
brief suggest content
content
content drug
content violenc
crude
crude humor
crude sexual
crude sexual content
disast
disast sequenc
disturb
disturb imag languag
disturb imag themat materi
disturb violent
drug
drug content languag
drug materi
drug sexual
edit
exclud
fantasi
fantasi action
fantasi action violenc
fi
fi action
fi action violenc
fi action violenc brief
fi action violenc languag
frighten
frighten imag
frighten imag brief
gener
grisli
humor
humor brief
humor languag drug
ima

In [13]:
# Abhishek Thakur
# https://www.kaggle.com/abhishek/beating-the-benchmark/code

svd = TruncatedSVD()
    
scl = StandardScaler()
    
svm_model = SVC(random_state=1)
    
clf = pipeline.Pipeline([('svd', svd), ('scl', scl), ('svm', svm_model)])
    
    
param_grid = {'svd__n_components' : [10, 15, 20, 40], 'svm__C': [1, 9,10, 12, 100], 'svm__tol': [0.0001], 
                'svm__decision_function_shape': ['ovr']}
    
model = grid_search.GridSearchCV(estimator = clf, param_grid=param_grid, scoring='accuracy', verbose=10, n_jobs=-1, iid=True, refit=True, cv=4)
                                     

model.fit(X_train, Y_train)
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
    
    
best_model = model.best_estimator_
    
print best_model
#best_model.fit(Xtrain,Y_train)
#preds = best_model.predict(X_test)
    
print(" Test Accuracy: %.2f%%" % (accuracy_score(Y_test, best_model.predict(X_test)) * 100.0))
print(" Train Accuracy: %.2f%%" % (accuracy_score(Y_train, best_model.predict(X_train)) * 100.0))    

Fitting 4 folds for each of 20 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    9.8s finished


Best score: 0.529
Best parameters set:
	svd__n_components: 40
	svm__C: 1
	svm__decision_function_shape: 'ovr'
	svm__tol: 0.0001
Pipeline(steps=[('svd', TruncatedSVD(algorithm='randomized', n_components=40, n_iter=5,
       random_state=None, tol=0.0)), ('scl', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svm', SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=1, shrinking=True,
  tol=0.0001, verbose=False))])
 Test Accuracy: 56.67%
 Train Accuracy: 72.21%


In [16]:
#Select important features with Random Forest
'''
clf = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1, max_depth=1)

# Train the classifier
clf.fit(train_df[train_df.columns.difference(['Category'])], train_df['Category'])

feat_labels = list(train_df.columns.values)

# Print the name and gini importance of each feature
#for feature in zip(feat_labels, clf.feature_importances_):
#    print(feature)
    
sfm = SelectFromModel(clf, threshold=0.005)

# Train the selector
sfm.fit(X_train, Y_train)

cols = []

for feature_list_index in sfm.get_support(indices=True):
    print(feat_labels[feature_list_index])
    cols.append(feat_labels[feature_list_index])
    
X_important_train = sfm.transform(X_train)

print X_important_train.shape


#if 'production_year' in cols:
#    cols.remove('production_year')

#X_train = X_train[cols]
#X_test = X_test[cols]
'''

"\nclf = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1, max_depth=1)\n\n# Train the classifier\nclf.fit(train_df[train_df.columns.difference(['Category'])], train_df['Category'])\n\nfeat_labels = list(train_df.columns.values)\n\n# Print the name and gini importance of each feature\n#for feature in zip(feat_labels, clf.feature_importances_):\n#    print(feature)\n    \nsfm = SelectFromModel(clf, threshold=0.005)\n\n# Train the selector\nsfm.fit(X_train, Y_train)\n\ncols = []\n\nfor feature_list_index in sfm.get_support(indices=True):\n    print(feat_labels[feature_list_index])\n    cols.append(feat_labels[feature_list_index])\n    \nX_important_train = sfm.transform(X_train)\n\nprint X_important_train.shape\n\n\n#if 'production_year' in cols:\n#    cols.remove('production_year')\n\n#X_train = X_train[cols]\n#X_test = X_test[cols]\n"

In [14]:
cm = confusion_matrix(Y_test, best_est.predict(X_test))

# Transform to df for easier plotting
cm_df = pd.DataFrame(cm,
                     index = ['1','2','3','4','5','6','7','8','9'], 
                     columns = ['1','2','3','4','5','6','7','8','9'])

plt.figure(figsize=(8,5))
sns.heatmap(cm_df, annot=True, fmt="d")
#plt.title('SVM Linear Kernel \nAccuracy:{0:.3f}'.format(accuracy_score(y_test, y_pred)))
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

NameError: name 'best_est' is not defined